In [8]:
import requests # om queries mee te berekenen
import pandas as pd # om dataset mee in te laden
from python_tsp.heuristics import solve_tsp_simulated_annealing # om optimale route mee te berekenen
import numpy as np # voor array berekenen

In [9]:
df = pd.read_csv("dataset.csv", sep=";")
df.head()

,huisnummer,postcode,woonplaats,lat,lon
0,6,8266JC,Kampen,52.539016,5.931903
1,205,8021AR,Zwolle,52.515551,6.107565
2,138,7415DD,Deventer,52.268968,6.163495
3,60,8261LP,Kampen,52.542804,5.927727
4,65,8031VV,Zwolle,52.529549,6.092312


In [11]:
for current_woonplaats in df['woonplaats'].unique():
    
    # haal alleen de locaties van de huidige woonplaats op
    temp_df = df[df['woonplaats'] == current_woonplaats].sample(10).copy() # maak een kopie, met maximaal 150 punten
    temp_df['rijnummer'] =  np.arange(temp_df.shape[0]) # voeg een rijnummer toe
    
    # uitrekenen van coordinaat paren, van losse kolommen naar 1 lange string
    # dus van:
    # lat, lon
    # 1234, 5678
    # 9102, 3456
    # naar:
    # 1234,5678;9102,3456
    coords = ""
    for coordpair in temp_df[['lat','lon']].astype(str).to_dict("records"):
        coords += coordpair['lon'] + "," + coordpair['lat'] + ";"
        
    # haal de laatste ; weg
    coords = coords[:-1]
    
    # doe een api call naar de OSRM-backend instantie en haal een matrix van deze routegegevens op
    distance_matrix = np.array(requests.get("http://localhost:5000/table/v1/driving/"+ coords).json()['durations'])
    
    # los het probleem op met behulp van de tsp_solver
    optimized_order, total_duration = solve_tsp_simulated_annealing(distance_matrix)
    
    # voeg de volgorde toe als kolom aan de orginele dataset
    for index, location in enumerate(optimized_order):
        temp_df.loc[temp_df['rijnummer'] == location, 'volgorde'] = index
        
    # sorteer de volgorde met de nieuwe gegevens om een routebook te maken   
    temp_df.sort_values(by="volgorde", inplace=True)
    
        
    temp_df.to_csv("regulier_" + current_woonplaats + ".csv", sep=";", index=False)
        
    print("De totale reistijd van deze route is ", round(total_duration/60), "uur, dit is de route voor:", current_woonplaats)

De totale reistijd van deze route is  24 uur, dit is de route voor: Kampen
De totale reistijd van deze route is  61 uur, dit is de route voor: Zwolle
De totale reistijd van deze route is  49 uur, dit is de route voor: Deventer
